In [0]:
import torch  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
DEVICE 

device(type='cuda')

In [0]:
EPOCH = 300
BATCH_SIZE = 128


In [6]:
#CIFAR 10 Dataset 사용 
image_download_url = "/content/gdrive/My Drive/Colab Notebooks/pytorch/cnn"

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root = image_download_url,
        train=True,
        download = True,
        transform = transforms.Compose([
                                        transforms.RandomCrop(32,padding=4),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5,0.5,0.5),
                                                             (0.5,0.5,0.5))
        ])), ## dataset 그거까지
        batch_size = BATCH_SIZE,
        shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root = image_download_url,
        train=False,
        transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,0.5,0.5)
                                      ,(0.5,0.5,0.5))
        ])
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

Files already downloaded and verified


In [0]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [0]:
## 모델정의 
## 이미지 -> 컨볼루션 + 배치정규화 -> basicblock 통과하고 -> 평균 풀링과 신경망 거쳐 예측 출력 
## basicblock = self.make_layer() 함수를 통해 하나의 모듈로 객체화 된다 

class ResNet(nn.Module):
  def __init__(self,num_classes = 10):
    super(ResNet, self).__init__()

    self.in_planes = 16 #layer1 층이 입력받는 채널의 개수가 16이여서 16으로 초기화 

    self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=1,padding=1,bias=False) ## -> 통과후 16*32*32
    self.bn1 = nn.BatchNorm2d(16)
    self.layer1 = self._make_layer(16,2,stride=1) ## 16*32*32
    self.layer2 = self._make_layer(32,2,stride=2) ## 32*16*16
    self.layer3 = self._make_layer(64,2,stride=2) ## 64*8*8
    self.linear = nn.Linear(64,out_features=num_classes)

  def _make_layer(self, planes, num_block, stride):
    print("in_make_layer")
    strides = [stride] + [1]*(num_block-1) ## ??뭐지 이거 
    print("planes  {} num_block : {} stride : {}".format(planes,num_block,stride))
    layers = []

    for stride in strides : 
      layers.append(BasicBlock(in_planes=self.in_planes,planes=planes,stride=stride))
      self.in_planes = planes
    
    return nn.Sequential(*layers)
  
  def forward(self,x):
    #print("in_Restent")
    out = F.relu(self.bn1(self.conv1(x)))
    #print("first _finish")
    out = self.layer1(out)
    #print("layer1")
    out = self.layer2(out)
    #print("layer2")
    out = self.layer3(out)
    #print("layter3")
    out = F.avg_pool2d(out,8)
    out = out.view(out.size(0),-1)##1차원으로 쫙 피고

    out = self.linear(out)

    return out

In [23]:
model = ResNet().to(DEVICE) ## GPU로 보내고 
optimizer = optim.SGD(model.parameters(),lr=0.1,momentum=0.9,weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer,step_size=50,gamma=0.1)
##optim.lr_scheduler.StepLR == 학습이 진행되면서 학습률을 점점 낮춰서 정교하게 최적화 하는것 
## step_size만큼 호출 되었을 때 학습률에 0.1값을 곱한다 
print(model)

in_make_layer
planes  16 num_block : 2 stride : 1
in_make_layer
planes  32 num_block : 2 stride : 2
in_make_layer
planes  64 num_block : 2 stride : 2
ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [0]:
def train(model, train_loader, optimizer, epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(DEVICE), target.to(DEVICE)
      optimizer.zero_grad()
      output = model(data)
      loss = F.cross_entropy(output, target)
      loss.backward()
      optimizer.step()

def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():##평가 과정에서는 기울기 계산 필요없음 
    for data, target in test_loader:
      data,target = data.to(DEVICE),target.to(DEVICE)
      output = model(data)

      ## 배치 오차 합산 
      test_loss += F.cross_entropy(output,target,reduction='sum').item()

      ## 가장 높은 값ㅇ르 가진 인덱스가 바로 예측 값 
      ## output.max() -> 2개의 값 출력 , 가장 큰 값과 인덱스 
      ## .eq() -> 값이 일치하는지 확인하는 것 
      pred = output.max(1,keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100.*correct/len(test_loader.dataset)
  return test_loss, test_accuracy

In [0]:
for epoch in range(1,EPOCH+1):
  scheduler.step() ##학습률을 조금 낮춰주는 단계
  
  train(model,train_loader,optimizer,epoch)
  test_loss ,test_accuracy = evaluate(model,test_loader)
  print('[{}] test loss : {:.4f}, accuracy : {:.2f}%'.format(epoch,test_loss,test_accuracy))

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1] test loss : 1.4112, accuracy : 49.11%
[2] test loss : 1.1272, accuracy : 61.18%
[3] test loss : 1.0150, accuracy : 64.47%
[4] test loss : 0.9553, accuracy : 68.12%
[5] test loss : 0.8945, accuracy : 69.20%
[6] test loss : 0.9682, accuracy : 69.68%
[7] test loss : 0.8160, accuracy : 72.34%
[8] test loss : 0.9641, accuracy : 70.25%
[9] test loss : 0.7171, accuracy : 75.15%
[10] test loss : 0.8435, accuracy : 72.20%
[11] test loss : 0.6762, accuracy : 77.05%
[12] test loss : 0.7674, accuracy : 75.19%
[13] test loss : 0.9534, accuracy : 70.22%
[14] test loss : 1.0972, accuracy : 68.34%
[15] test loss : 0.7396, accuracy : 75.57%
[16] test loss : 0.8477, accuracy : 73.27%
[17] test loss : 0.6437, accuracy : 77.42%
[18] test loss : 0.6263, accuracy : 78.33%
[19] test loss : 0.8175, accuracy : 73.95%
[20] test loss : 0.6596, accuracy : 77.49%
[21] test loss : 0.8940, accuracy : 73.01%
[22] test loss : 1.2606, accuracy : 65.12%
[23] test loss : 0.8182, accuracy : 74.79%
[24] test loss : 1.0